# 概述
LangChain提供了两种定义工具的方式：

第一种：使用`@tool`装饰器（自定义工具的最简单方式）

装饰器默认使用`函数名称`作为工具名称，但可以通过参数 `name_or_callable` 来修改默认值。

同时，装饰器默认使用函数的 `文档字符串` 作为工具的描述 ，因此函数必须提供文档字符串。


第二种：使用`StructuredTool.from_function`方法来定义工具。
这类似于 @tool 装饰器，但允许更多配置和同步/异步实现的规范。



Tool的几个常用属性说明：

|属性|类型|描述|
|--|--|--|
|name|str|工具名称，必选的 ，在提供给LLM或Agent的工具集中必须是唯一的。
|description|str|工具描述，可选但建议，方便LLM或Agent理解工具的作用以确定是否要使用。
|args_schema|Pydantic BaseModel|可选但建议，可用于提供更多信息（例如，few-shot示例）或验证预期参数。
|return_direct|bool|可选，仅对Agent相关。当为True时，工具调用结果直接返回给用户；当为False时，工具调用结果将返回给Agent（LLM）。

# 自定义工具代码实现
## 方式1：@tool 装饰器
举例1：tool的默认参数

In [3]:
import json

from langchain_core.tools import tool


@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers together."""
    return a + b


print(f"name: {add_numbers.name}")
print(f"description: {add_numbers.description}")
print(f"args: {add_numbers.args}")
print(f"return_direct: {add_numbers.return_direct}")

name: add_numbers
description: Add two numbers together.
args: {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}
return_direct: False


举例2：通过@tool的参数设置修改默认值

In [4]:
from pydantic import BaseModel, Field
from langchain_core.tools import tool


# 工具参数描述
class FieldInfo(BaseModel):
    a: int = Field(description="第一个参数")
    b: int = Field(description="第二个参数")


# 定义工具
@tool(
    name_or_callable="add_two_numbers2",
    description="Add two numbers together2.",
    return_direct=True,
    args_schema=FieldInfo
)
def add_numbers(a: int, b: int) -> int:
    """Add two numbers together."""
    return a + b


print(f"name: {add_numbers.name}")
print(f"description: {add_numbers.description}")
print(f"args: {add_numbers.args}")
print(f"return_direct: {add_numbers.return_direct}")

name: add_two_numbers2
description: Add two numbers together2.
args: {'a': {'description': '第一个参数', 'title': 'A', 'type': 'integer'}, 'b': {'description': '第二个参数', 'title': 'B', 'type': 'integer'}}
return_direct: True


## 方式2：StructuredTool的from_function()
`StructuredTool.from_function` 类方法提供了比 `@tool` 装饰器更多的可配置性，而无需太多额外的代码。

举例1：

In [5]:
from langchain_core.tools import StructuredTool


# 自定义函数
def search_function(query: str):
    return "LangChain"


# 将自定义函数包装为工具
search1 = StructuredTool.from_function(
    func=search_function,
    name="Search",
    description="useful for when you need to answer questions about current events"
)

print(f"name = {search1.name}")
print(f"description = {search1.description}")
print(f"args = {search1.args}")
# 调用工具
search1.invoke("hello")

name = Search
description = useful for when you need to answer questions about current events
args = {'query': {'title': 'Query', 'type': 'string'}}


'LangChain'

举例2：

In [6]:
from langchain_core.tools import StructuredTool
from pydantic import Field, BaseModel


# 工具参数描述
class FieldInfo(BaseModel):
    query: str = Field(description="要检索的关键词")


def search_function(query: str):
    return "LangChain"


search1 = StructuredTool.from_function(
    func=search_function,
    name="Search",
    description="useful for when you need to answer questions about current events",
    args_schema=FieldInfo,
    return_direct=True,
)
print(f"name = {search1.name}")
print(f"description = {search1.description}")
print(f"args = {search1.args}")
print(f"return_direct = {search1.return_direct}")
search1.invoke("hello")

name = Search
description = useful for when you need to answer questions about current events
args = {'query': {'description': '要检索的关键词', 'title': 'Query', 'type': 'string'}}
return_direct = True


'LangChain'

# 工具调用结合LLM
## 举例1：大模型分析是否调用工具

In [7]:
#1.导入相关依赖
from langchain_community.tools import MoveFileTool
from langchain_core.messages import HumanMessage
from langchain_core.utils.function_calling import convert_to_openai_function
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
# 2.定义LLM模型
chat_model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

D:\develop\miniconda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [8]:
# 3.定义工具
tools = [MoveFileTool()]
# 4.这里需要将工具转换为openai函数，后续再将函数传入模型调用
functions = [convert_to_openai_function(t) for t in tools]
# print(functions[0])
# 5. 提供大模型调用的消息列表
messages = [HumanMessage(content="将文件a移动到桌面")]
# 6.模型使用函数
# 说明：模型调用工具时，不接收tools，只接收functions，因此需要对tool做一个转换
response = chat_model.invoke(
    input=messages,
    functions=functions
)
print(response)

content='' additional_kwargs={'function_call': {'arguments': '{"source_path":"a","destination_path":"/Users/YourUsername/Desktop/a"}', 'name': 'move_file'}, 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 76, 'total_tokens': 103, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-CUVzKoykOal13Fg6DqOhLNngiLgW1', 'finish_reason': 'function_call', 'logprobs': None} id='lc_run--da2bf4e2-e912-46af-b220-17d91c29e0a4-0' usage_metadata={'input_tokens': 76, 'output_tokens': 27, 'total_tokens': 103, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


作为对比，让模型调用工具，传递用户输入，看模型是否会调用不相关的工具呢？

In [9]:
response = chat_model.invoke(
    input=[HumanMessage("今天西安的天气怎么样呢？")],
    functions=functions
)
print(response)

content='抱歉，我无法提供实时天气信息。建议你查看天气预报网站或使用天气应用程序获取西安今天的天气情况。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 77, 'total_tokens': 108, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-CUW25OF8GqwleojpmGfUB6tEe2h9L', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--24f81754-26d0-4c3f-8118-f4bd6ec4a12d-0' usage_metadata={'input_tokens': 77, 'output_tokens': 31, 'total_tokens': 108, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


可以看到模型输出结果中，没有`function_call`部分数据，`content`有输出了（没有合适工具可调用的相关提示）

## 工具调用说明
现在模型调用工具有两种情况：

**情况1：模型调用工具**

如果模型认为需要调用工具（如 MoveFileTool ），返回的 `message` 会包含：
* content：通常为空，因为模型选择调用工具，而非生成文本回复
* additional_kwargs：会包含工具调用的详细信息
```json
AIMessage(
    content='', # 无自然语言回复
    additional_kwargs={
        'function_call': {
            'name': 'move_file', # 工具名称
            'arguments':
            '{"source_path":"a","destination_path":"/Users/YourUsername/Desktop/a"}' # 工具参数
        }
    }
)
```

**情况2：大模型不调用工具**

如果模型认为无需调用工具（例如用户输入与工具无关），返回的 `message` 会是一段普通文本回复：
```python
AIMessage(
    content='我没有找到需要移动的文件。', # 自然语言回复
    additional_kwargs={'refusal': None} # 无工具调用
)
```

## 举例2：确定工具并调用

In [12]:
# 定义LLM模型
chat_model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# 定义工具
tools = [MoveFileTool()]
# 将工具转换为openai函数
functions = [convert_to_openai_function(t) for t in tools]
# 提供消息列表
messages = [HumanMessage(content="将当前目录下的a.txt文件移动到我指定的目录下（D:\\tmp）")]
# 模型调用
response = chat_model.invoke(
    input=messages,
    functions=functions
)
print(response)

content='' additional_kwargs={'function_call': {'arguments': '{"source_path":"a.txt","destination_path":"D:\\\\tmp\\\\a.txt"}', 'name': 'move_file'}, 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 89, 'total_tokens': 116, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-CUWDz1JGINpmP8X6hT4v8SCUeADJI', 'finish_reason': 'function_call', 'logprobs': None} id='lc_run--c8fe0bae-fade-4ae2-9acd-d888b887b519-0' usage_metadata={'input_tokens': 89, 'output_tokens': 27, 'total_tokens': 116, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


**（1）检查是否需要调用工具**

In [21]:
# 是否需要调用工具
is_invoke = False
if "function_call" in response.additional_kwargs:
    tool_name = response.additional_kwargs["function_call"]["name"]
    tool_args = response.additional_kwargs["function_call"]["arguments"]
    is_invoke = True
    print(f"调用工具: {tool_name}，参数: {tool_args}")
else:
    print(f"模型回复：{response.co}")

调用工具: move_file，参数: {"source_path":"a.txt","destination_path":"D:\\tmp\\a.txt"}


**（2）实际执行工具**

In [23]:
if is_invoke:
    tool = MoveFileTool()
    result = tool.run(json.loads(tool_args))
    print(f"工具执行结果：{result}")

工具执行结果：File moved successfully from a.txt to D:\tmp\a.txt.
